# Interactive Black-Scholes Option Pricing Model

> **Buy-Side Perspective:**
> Valuation of options is critical not just for derivatives traders but for credit analysts assessing convertible bonds, warrants attached to debt, or the implied volatility of a firm's equity as a signal of distress (Merton Model).

## Overview
This notebook implements the Black-Scholes-Merton model for pricing European call and put options. It includes interactive widgets to see how changing inputs (like volatility or time to expiry) affects the option price and the "Greeks."


In [ ]:
import numpy as np
import scipy.stats as si
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def black_scholes(S, K, T, r, sigma, option_type='call'):
    '''
    S: Spot price
    K: Strike price
    T: Time to maturity (in years)
    r: Risk-free interest rate (decimal)
    sigma: Volatility of underlying asset (decimal)
    option_type: 'call' or 'put'
    '''
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    if option_type == 'call':
        price = (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
        delta = si.norm.cdf(d1)
    else:
        price = (K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0) - S * si.norm.cdf(-d1, 0.0, 1.0))
        delta = -si.norm.cdf(-d1)
        
    gamma = si.norm.pdf(d1) / (S * sigma * np.sqrt(T))
    theta = -(S * si.norm.pdf(d1) * sigma) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * si.norm.cdf(d2) # Simplified for call
    if option_type == 'put':
        theta = -(S * si.norm.pdf(d1) * sigma) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * si.norm.cdf(-d2)
        
    vega = S * np.sqrt(T) * si.norm.pdf(d1)
    
    return price, delta, gamma, theta, vega

# Example
price, delta, gamma, theta, vega = black_scholes(100, 100, 1, 0.05, 0.2, 'call')
print(f"Call Price: {price:.2f}")


## Interactive Pricing
Adjust the sliders to see how the option price changes.


In [ ]:
def interactive_pricer(S, K, T, r, sigma, option_type):
    price, delta, gamma, theta, vega = black_scholes(S, K, T, r/100, sigma/100, option_type)
    
    print(f"Option Price: ${price:.2f}")
    print("-" * 20)
    print(f"Delta: {delta:.4f}")
    print(f"Gamma: {gamma:.4f}")
    print(f"Theta: {theta:.4f}")
    print(f"Vega:  {vega:.4f}")

widgets.interact(interactive_pricer, 
                 S=widgets.FloatSlider(min=10, max=200, step=1, value=100, description='Spot Price'),
                 K=widgets.FloatSlider(min=10, max=200, step=1, value=100, description='Strike Price'),
                 T=widgets.FloatSlider(min=0.1, max=5, step=0.1, value=1.0, description='Time (Years)'),
                 r=widgets.FloatSlider(min=0, max=20, step=0.25, value=5.0, description='Risk-Free %'),
                 sigma=widgets.FloatSlider(min=1, max=100, step=1, value=20.0, description='Volatility %'),
                 option_type=widgets.Dropdown(options=['call', 'put'], value='call', description='Type'));
